In [ ]:
%pip install -r requirements.txt

### Language Model Configuration

This cell sets up the configuration for the LLM (Language Model), defining the model details, API endpoint, and authentication. The `cache_seed` is set to `None` to disable result caching.

In [1]:
gemma = {
    "config_list": [
        {
            "model": "lmstudio-community/DeepSeek-R1-Distill-Llama-8B-GGUF/DeepSeek-R1-Distill-Llama-8B-Q4_K_M.gguf:0",
            "base_url": "http://localhost:1234/v1",
            "api_key": "lm-studio",
            "price": [0.001, 0.001],  # Add custom pricing here (per 1k prompt/completion tokens),
            "stream": False
        },
    ],
    "cache_seed": None,  # Disable caching.
}

### Importing Required Modules

- `Image` and `display` from `IPython.display` for displaying images in the notebook.
- The `autogen` library, which provides the tools for creating agents and handling conversations.
- `LocalCommandLineCodeExecutor` from `autogen.coding`, a utility that allows the assistant agent to execute local shell commands for coding tasks.


In [2]:
from IPython.display import Image, display

import autogen
from autogen.coding import LocalCommandLineCodeExecutor

### Creating the Assistant and User Proxy Agents

Setting up two agents:
- **AssistantAgent (`assistant`)**:
  - Uses the LLM configuration (`gemma`).
  - Acts as the main AI assistant, capable of handling tasks and responding based on the LLM's capabilities.

- **UserProxyAgent (`user_proxy`)**:
  - Represents a user agent that interacts with the assistant.
  - `human_input_mode` is set to `"NEVER"`, meaning it won't require direct user input.
  - The agent can auto-reply up to 10 times consecutively before requiring further input or ending the task.
  - The `is_termination_msg` lambda function checks for specific keywords (`"TERMINATE"` or `"COMPLETED"`) to decide when to stop the conversation.
  - `code_execution_config` specifies the use of `LocalCommandLineCodeExecutor`, which allows the assistant to run shell commands in a local working directory (`coding`).

In [3]:
# create an AssistantAgent named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=gemma, 
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", "").upper() or "COMPLETED" in x.get("content", "").upper(),
    code_execution_config={
        # the executor to run the generated code
        "executor": LocalCommandLineCodeExecutor(work_dir="coding"),
    },
)

### Starting the Interaction

Initiating a chat between the `user_proxy` and the `assistant`:
- The `user_proxy` sends an initial message to the `assistant`: `"How many r are in rasberry?"`, unintentionally misspelling "raspberry."
- The `summary_method` is set to `"reflection_with_llm"`, meaning the response summary will leverage the LLM for improved reflection and analysis.

The result of the conversation (`chat_res`) will contain the exchange between the two agents, including any code execution if requested by the assistant.

In [4]:
# the assistant receives a message from the user_proxy, which contains the task description
chat_res = user_proxy.initiate_chat(
    assistant,
    message="""How many r are in rasberry?""",
    summary_method="reflection_with_llm",
)

user_proxy (to assistant):

How many r are in rasberry?

--------------------------------------------------------------------------------
assistant (to user_proxy):

<think>
Okay, so I'm trying to figure out how many 'r's are in "rasberry." Let me break this down step by step. First, I'll write down the word: R-A-S-B-E-R-R-Y. Hmm, let's count each letter one by one.

Starting with the first letter, it's 'R'—that's one 'r.' Then we have 'A,' which is not an 'r. Next is 'S,' also not an 'r. Moving on to 'B,' still no 'r there. The next letter is 'E,' so that's not an 'r either. Now, the fifth letter is another 'R'—so that's two 'r's now.

Wait, but then I see another 'R' in "rasberry"—is that correct? Let me double-check. Oh, wait, the word is spelled R-A-S-B-E-R-R-Y. So after 'E,' there are two 'R's before the 'Y.' That means we have three 'r's: one at the beginning, and then two more in the middle. Wait, no, that doesn't sound right because I'm seeing only three letters, but maybe I'm 